# Imports

In [1]:
import pandas as pd
import numpy as np

# Load and Display

In [3]:
# Load the dataset
data = pd.read_csv('Playstore_final.csv')

# Display the first few rows of the dataset
print("Initial Data:")
print(data.head())

C:\Users\fathi\AppData\Local\Temp\ipykernel_8976\3349051028.py:2: DtypeWarning: Columns (7,12,29,30,33,35,37,38,40,42,43,44,46,47,48,49,50,52,53) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('Playstore_final.csv')


Initial Data:
                  App Name                                             App Id  \
0     Logistics Management              com.eniseistudio.logistics_management   
1  Estados Unidos Noticias               com.eniseistudio.news.estados_unidos   
2         Dental Assistant                  com.eniseistudio.dental_assistant   
3        Medical Assistant          com.eniseistudio.course.medical_assistant   
4  Business Administration  com.eniseistudio.majors.course.business_admini...   

           Category    Rating  Rating Count Installs  Minimum Installs  Free  \
0         Education  4.090909          66.0  10,000+           10000.0  True   
1  News & Magazines  4.000000           8.0   1,000+            1000.0  True   
2         Education  3.866667          15.0  10,000+           10000.0  True   
3         Education  4.000000          18.0   5,000+            5000.0  True   
4         Education  4.023256          86.0  50,000+           50000.0  True   

   Price Currency 

# Convert 'Installs' and 'Minimum Installs' columns to numeric

In [4]:
data['Installs'] = data['Installs'].str.replace('+', '').str.replace(',', '').astype(float)
data['Minimum Installs'] = data['Minimum Installs'].astype(float)

C:\Users\fathi\AppData\Local\Temp\ipykernel_8976\3606521241.py:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  data['Installs'] = data['Installs'].str.replace('+', '').str.replace(',', '').astype(float)


# Handle Missing Values

In [5]:
data.fillna({
    'Rating': data['Rating'].mean(),
    'Rating Count': 0,
    'Size': 'Varies with device',
    'Developer Website': 'N/A',
    'Privacy Policy': 'N/A'
}, inplace=True)

# Convert 'Released' and 'Last update' columns to datetime

In [6]:
data['Released'] = pd.to_datetime(data['Released'], format='%d-%b-%y', errors='coerce')
data['Last update'] = pd.to_datetime(data['Last update'], format='%d-%b-%y', errors='coerce')

# Remove leading and trailing whitespaces from string columns

In [7]:
data = data.applymap(lambda x: x.strip() if isinstance(x, str) else x)

# Remove any duplicate rows

In [8]:
data.drop_duplicates(inplace=True)

# Replace 'N/A' with NaN for better handling in future analysis

In [9]:
data.replace('N/A', np.nan, inplace=True)

# Drop columns with all missing values

In [11]:
data.dropna(how='all', axis=1, inplace=True)

# Display Info

In [12]:
print("\nCleaned Data Info:")
print(data.info())


Cleaned Data Info:
<class 'pandas.core.frame.DataFrame'>
Int64Index: 426413 entries, 0 to 450791
Data columns (total 54 columns):
 #   Column                 Non-Null Count   Dtype         
---  ------                 --------------   -----         
 0   App Name               426400 non-null  object        
 1   App Id                 426413 non-null  object        
 2   Category               426400 non-null  object        
 3   Rating                 426413 non-null  float64       
 4   Rating Count           426413 non-null  float64       
 5   Installs               426322 non-null  float64       
 6   Minimum Installs       426400 non-null  float64       
 7   Free                   426321 non-null  object        
 8   Price                  426321 non-null  float64       
 9   Currency               426321 non-null  object        
 10  Size                   426413 non-null  object        
 11  Minimum Android        425360 non-null  object        
 12  Developer Id           4

# Save cleaned data to a new CSV file

In [13]:
data.to_csv('cleaned_dataset.csv', index=False)